In [1]:
# importing required libraries

import pandas as pd
import numpy as np
import json
import re
import os
import torch
from transformers import AutoTokenizer, AutoModel

/home/avd6/utils/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# creating the class

class ContrieverCB:
    def __init__(self):
        self.embeddings = {}
    
    
    def clean(self, text: str) -> str:
        """
        Function to remove newline from text.
        :param text: input string
        :return: string without newline
        """
        new_text = re.sub('\n', '', text)
        return new_text
    
    
    def mean_pooling(self, token_embeddings, mask):
        """
        Function to be used after model is applied to tokenized text to generate embeddings.
        Used in the HuggingFace version.
        :param token_embeddings: output of model
        :param mask: attention mask of the tokens
        :return: tensors of the text
        """
        token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
        sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
        return sentence_embeddings
    
    
    def generate_embeddings(self, path_to_json: str, path_to_output: str) -> None:
        """
        Function takes input json filepath, generates numpy embeddings of the file and
        saves them at the given output filepath.
        :param path_to_json: input filepath
        :param path_to_output: output filepath
        :return: None
        """
        #tokenizer = AutoTokenizer.from_pretrained('facebook/contriever-msmarco')
        tokenizer = AutoTokenizer.from_pretrained('/mnt/project/chatbotai/asmita/output/facebook-contriever-msmarco-2023-02-12_18-47-40/final')
        #model = AutoModel.from_pretrained('facebook/contriever-msmarco')
        model = AutoModel.from_pretrained('/mnt/project/chatbotai/asmita/output/facebook-contriever-msmarco-2023-02-12_18-47-40/final')
        
        # open and read the input json file
        file = open(path_to_json)
        json_data = json.load(file)
        
        n = int(len(json_data)/100)
        embeddings_list = []
        
        # take 100 units at a time and process it
        for k in range(n):
            if k==n:
                start = k*100
                end = (list(json_data.keys())[-1])
            else:
                start = k*100
                end = k*100+99
                
            for i in range(start, end):
                text = json_data[i]['text']
                text = self.clean(text)
                tokenized_text = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
                output = model(**tokenized_text)
                embeddings = self.mean_pooling(output[0], tokenized_text['attention_mask'])
                embeddings_np = embeddings.detach().numpy()
                embeddings_list.append(embeddings_np)
                
        # convert embeddings list to numpy array
        embeddings_array = np.array(embeddings_list)
        
        # reshape the numpy array
        x = embeddings_array.shape[0]
        y = embeddings_array.shape[2]
        embeddings_array.reshape((x,y))
        
        # save the embeddings in a numpy file
        # filename = path_to_json.split('\\')[-1].split('.')[0]
        # filepath = os.path.join(path_to_output, filename)
        filepath = path_to_output
        
        # saving the embeddings as a numpy file in the destination folder
        np.save(filepath, embeddings_array)
        
        # saving the embeddings into a dictionary
        self.embeddings[path_to_json] = embeddings_array
        
    
    def retrieve_topk(self, search_string: str, path_to_json: str, k: int):
        """
        Function takes json data as input and returns the topk relative to the data
        :param search_string: query to match and retrieve
        :param path_to_json: filepath of data to search
        :param k: number of embeddings to retrieve
        :return: top k units relative to the input file
        """
        
        # check if the embeddings are loaded in dictionary already
        if not (path_to_json in self.embeddings):
            
            # changing .json to .npy
            filename = os.path.splitext(path_to_json)[0]
            path_to_npy = filename + '.npy'
            
            # check if .npy exists and load into dictionary
            if os.path.exists(path_to_npy):
                self.embeddings[path_to_json] = np.load(path_to_npy)
            else:
                # .npy doesn't exist, so generate embeddings
                self.generate_embeddings(path_to_json, path_to_npy)
        
        # convert numpy embeddings to tensors
        embeddings = torch.from_numpy(self.embeddings[path_to_json])
        
        # convert query to tensor
        #tokenizer = AutoTokenizer.from_pretrained('facebook/contriever-msmarco')
        tokenizer = AutoTokenizer.from_pretrained('/mnt/project/chatbotai/asmita/output/facebook-contriever-msmarco-2023-02-12_18-47-40/final')
        #model = AutoModel.from_pretrained('facebook/contriever-msmarco')
        model = AutoModel.from_pretrained('/mnt/project/chatbotai/asmita/output/facebook-contriever-msmarco-2023-02-12_18-47-40/final')
        
        tokenized_query = tokenizer(search_string, padding=True, truncation=True, return_tensors='pt')
        output_query = model(**tokenized_query)
        embedded_query = self.mean_pooling(output_query[0], tokenized_query['attention_mask'])
        
        # creating a dictionary of scores
        scores_list = []
        for i in range(len(embeddings)):
            score = embeddings[i]@embedded_query[0]
            score_np = score.detach().numpy()[0]
            scores_list.append([i, score_np])
            
        scores_df = pd.DataFrame(scores_list, columns=['ID', 'Score'])
        
        # retrieving top k scores
        topk_scores = scores_df.nlargest(k, 'Score')
        
        # retrieving the text data corresponding to the top k indices
        topk_context = {}
        
        json_file = open(path_to_json)
        json_data = json.load(json_file)
        
        for row in topk_scores.iterrows():
            ind = int(row[1]['ID'])
            text = json_data[ind]['text']
            topk_context[ind] = text
        
        return topk_context

In [12]:
# loading dataset
file = open("/mnt/project/chatbotai/asmita/non-public-datasets/cleaned_data/ece120_notes/ece120_notes_cleaned.json")
data = json.load(file)

In [23]:
filepath = "/mnt/project/chatbotai/asmita/non-public-datasets/cleaned_data/ece120_notes/ece120_notes_cleaned.json"

In [20]:
contriever = ContrieverCB()
query = "How does the design of a finite state machine differ from that of a counter?"
topk = contriever.retrieve_topk(query, filepath, 5)

In [21]:
# test with original model
topk

{98: '3.2 Finite State Machine Design Examples, Part I 99 3.2.11 Impact of the State Representation What happens if we choose a bad representation? For the same FSM —the two-bit Gray code counter with start and stop inputs—the table below shows a poorly chosen mappin g from states to internal state repre- sentation. Below the table is a diagram of an implementation using that representation. Verifying that the implementation’s behavior is correct is left as an exercise for the det ermined reader. state S2S1S0 state S2S1S0 COUNT A 000 HALT A 111 COUNT B 101 HALT B 110 COUNT C 011 HALT C 100 COUNT D 010 HALT D 001 D QQ S0D QQ S1D QQ S2 G (go button)a controllable two−bit counter (with poorly chosen state values) Z1 Z0CLOCK H(halt button)',
 90: '3.2 Finite State Machine Design Examples, Part I 91 Notice that the equations for S+ 2andS+ 1share a com- mon term, S1S0. This design does not allow much choice in developing good equations for the next-state logic, but some designs may enable yo

In [26]:
contriever = ContrieverCB()
query = "How does the design of a finite state machine differ from that of a counter?"
topk = contriever.retrieve_topk(query, filepath, 10)

Some weights of the model checkpoint at /mnt/project/chatbotai/asmita/output/facebook-contriever-msmarco-2023-02-12_18-47-40/final were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# test with finetuned model
topk

{88: '3.2 Finite State Machine Design Examples, Part I 89 ECE120: Introduction to Computer Engineering Notes Set 3.2 Finite State Machine Design Examples, Part I This set of notes uses a series of examples to illustrate design princip les for the implementation of ﬁnite state machines (FSMs) using digital logic. We begin with an overview of the design process for a digital FSM, from the development of an abstract model through the imple mentation of functions for the next-state variables and output signals. Our ﬁrst few examples cover only the c oncrete aspects: we implement several counters, which illustrate the basic process of translating a concr ete and complete state transition diagram into an implementation based on ﬂip-ﬂops and logic gates. We next con sider a counter with a number of states that is not a power of two, with which we illustrate the need fo r FSM initialization. We then consider the design process as a whole through a more gene ral example of a counter with multip